# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\scripts/training/bidding/model/bidding-1248000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, -1, -1, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H', 'insta_score': 0.6496},
 {'call': 'PASS', 'insta_score': 0.1713},
 {'call': 'X', 'insta_score': 0.1437}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.7673},
 {'call': '6H', 'insta_score': 0.2015}]

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(["PAD_START", "PASS", "1H", "X", "4H", "PASS", "PASS", "X"])
bid.bid

'PASS'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.7176}, {'call': '6H', 'insta_score': 0.183}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['x.AKJT8x.xxx.Kxx Jxxxx.x.QTxx.9xx AQ9.Q9xxx.9.QTxx KT8x.x.AKJ8x.AJ8',
 'x.AKJT8x.xxx.Kxx 98xxx.x.J98x.Q9x AQT.Qxxxx.Tx.ATx KJxx.9.AKQx.J8xx',
 'x.AKJT8x.xxx.Kxx Txxx.x.QJT9x.xxx KJ98.Q9xxx.K.Q9x AQxx.x.A8xx.AJT8',
 'x.AKJT8x.xxx.Kxx 98xxx.x.KQ98.9xx AKTx.Q9xxx.x.QTx QJx.x.AJTxx.AJ8x',
 'x.AKJT8x.xxx.Kxx A98xx.x.8xx.J8xx KQx.Q9xxx.K9.9xx JTxx.x.AQJTx.AQT',
 'x.AKJT8x.xxx.Kxx K9xxx.xx.QTx.Q9x A8x.Q9xxx.K8.JT8 QJTx..AJ9xx.Axxx',
 'x.AKJT8x.xxx.Kxx QTxxxx.xx.Tx.9xx AJ9.Q9xxx.Q9x.AT K8x..AKJ8x.QJ8xx',
 'x.AKJT8x.xxx.Kxx xxx.x.98xx.J9xxx AKT8.Q9xxx.KJT.T QJ9xx.x.AQx.AQ8x',
 'x.AKJT8x.xxx.Kxx QT9xxx.9x.x.QJTx AJ8x.Qxxxx.KT9.x Kx..AQJ8xx.A98xx',
 'x.AKJT8x.xxx.Kxx AT98xxx.x.8x.xxx QJx.Q9xxx.KJ.QJ8 Kx.x.AQT9xx.AT9x',
 'x.AKJT8x.xxx.Kxx xxx.x.J98x.QJxxx AJ8x.Q9xxxx.Ax.9 KQT9x..KQTx.AT8x',
 'x.AKJT8x.xxx.Kxx Q9xxx.Q.T8x.JTxx 8.9xxxx.KQJ9.A98 AKJTxx.x.Axx.Qxx',
 'x.AKJT8x.xxx.Kxx Txxx.Qxx.9x.QJxx AJxx.9xxx.KQJ.98 KQ98..AT8xx.ATxx',
 'x.AKJT8x.xxx.Kxx JT98x.9x.8.QJT9x Kxxx.Qxxx.AQT.xx AQx.x.KJ9xx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'])
print(sample_hands.shape[0])
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AttributeError: 'tuple' object has no attribute 'shape'

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [ ]:
hands = random_deal().split()
hands # N, E, S, W

In [ ]:
bidder_bots = [BotBid([False, False], hand, models, -1, -1, sampler, False) for hand in hands]

In [ ]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction